In [4]:
import shutil

shutil.move("/images.zip", "/content/images.zip")
print("Moved to /content/images.zip")



Moved to /content/images.zip


In [5]:
import zipfile

with zipfile.ZipFile("/content/images.zip", 'r') as z:
    z.extractall("/content/images")

In [6]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [7]:
TRAIN_DIR = '/content/images/images'

In [8]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(150, 150, 3)),
    # Rescale the image. Note the input shape is the desired size of the image: 150x150 with 3 bytes for color
    tf.keras.layers.Rescaling(1./255),
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(8, activation='softmax')
])

# Print the model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 148, 148, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 15, 15, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     3,211,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │         4,104 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,476,040 (13.26 MB)

 Trainable params: 3,476,040 (13.26 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
training_dataset = tf.keras.utils.image_dataset_from_directory(


   directory=TRAIN_DIR,
   image_size=(150,150),
   batch_size=32,
   label_mode='categorical',
   validation_split=0.15,
   subset='training',
   seed=42
    )

validation_dataset = tf.keras.utils.image_dataset_from_directory(
        directory=TRAIN_DIR,
        image_size=(150,150),
        batch_size=32,
        label_mode='categorical',
        validation_split=0.15,
        subset='validation',
        seed=42
    )

Found 23919 files belonging to 8 classes.
Using 20332 files for training.
Found 23919 files belonging to 8 classes.
Using 3587 files for validation.


In [10]:
val_batches = validation_dataset.cardinality().numpy()

test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

print("Validation batches:", validation_dataset.cardinality().numpy())
print("Test batches:", test_dataset.cardinality().numpy())


Validation batches: 91
Test batches: 22


In [11]:
import tensorflow as tf

def count_images(ds):
    total = 0
    for batch, labels in ds:
        total += batch.shape[0]   # number of images in this batch
    return total

print("Total TRAIN images:", count_images(training_dataset))
print("Total VALIDATION images:", count_images(validation_dataset))
print("Total TEST images:", count_images(test_dataset))


Total TRAIN images: 20332
Total VALIDATION images: 2883
Total TEST images: 704


In [12]:


# Optimize the datasets for training
SHUFFLE_BUFFER_SIZE = 8000
PREFETCH_BUFFER_SIZE = tf.data.AUTOTUNE

train_dataset_final = (training_dataset
                       .cache()
                       .shuffle(SHUFFLE_BUFFER_SIZE)
                       .prefetch(PREFETCH_BUFFER_SIZE)
                       )

validation_dataset_final = (validation_dataset
                            .cache()
                            .prefetch(PREFETCH_BUFFER_SIZE)
                            )

In [13]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',      # watch validation loss
    patience=3,              # stop after 3 epochs with no improvement
    restore_best_weights=True
)

In [14]:
model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
history = model.fit(
	train_dataset_final,
	epochs=35,
	validation_data=validation_dataset_final,
	callbacks = [early_stop]
)

Epoch 1/35
636/636 ━━━━━━━━━━━━━━━━━━━━ 169s 75ms/step - accuracy: 0.5069 - loss: 1.0803 - val_accuracy: 0.8741 - val_loss: 0.2859
Epoch 2/35
636/636 ━━━━━━━━━━━━━━━━━━━━ 21s 33ms/step - accuracy: 0.9022 - loss: 0.2517 - val_accuracy: 0.9761 - val_loss: 0.0789
Epoch 3/35
636/636 ━━━━━━━━━━━━━━━━━━━━ 21s 33ms/step - accuracy: 0.9717 - loss: 0.0846 - val_accuracy: 0.9743 - val_loss: 0.0735
Epoch 4/35
636/636 ━━━━━━━━━━━━━━━━━━━━ 21s 34ms/step - accuracy: 0.9805 - loss: 0.0578 - val_accuracy: 0.9840 - val_loss: 0.0547
Epoch 5/35
636/636 ━━━━━━━━━━━━━━━━━━━━ 22s 34ms/step - accuracy: 0.9861 - loss: 0.0411 - val_accuracy: 0.9830 - val_loss: 0.0627
Epoch 6/35
636/636 ━━━━━━━━━━━━━━━━━━━━ 22s 34ms/step - accuracy: 0.9861 - loss: 0.0476 - val_accuracy: 0.9792 - val_loss: 0.0571
Epoch 7/35
636/636 ━━━━━━━━━━━━━━━━━━━━ 22s 34ms/step - accuracy: 0.9880 - loss: 0.0351 - val_accuracy: 0.9840 - val_loss: 0.0533
Epoch 8/35
636/636 ━━━━━━━━━━━━━━━━━━━━ 21s 34ms/step - accuracy: 0.9879 - loss: 0.0347 -

In [15]:
test_loss, test_acc = model.evaluate(test_dataset)
print("Test accuracy:", test_acc)
print("Test loss:", test_loss)

22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 160ms/step - accuracy: 0.9926 - loss: 0.0234
Test accuracy: 0.9957386255264282
Test loss: 0.020320825278759003


In [16]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

y_true = []
y_pred = []

for images, labels in test_dataset:
    preds = model.predict(images)
    y_pred.extend(np.argmax(preds, axis=1))
    y_true.extend(np.argmax(labels.numpy(), axis=1))

cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=training_dataset.class_names))


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
Confusion Matrix:
[[80  0  0  0  0  0  0  0]
 [ 0 83  0  0  0  0  0  0]
 [ 0  0 75  0  0  0  0  0]
 [ 0  2  0 84  0  0  0  0]
 [ 0  0  0  0 85  0  